<a href="https://colab.research.google.com/github/nhuyen183/LungCancerSupportSystem/blob/master/BRFSS_preprocessingggg_fianlllll_18Mar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Installing Spark and dependencies
#Java 8
#Apache Spark with hadoop and
#Findspark (used to locate the spark in the system)
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

#Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [920 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,544 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,021 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,017 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,313 kB]
Hi

# Step 1: Define the problem
What sorts of people were likely to have lung cancer?

# Step 2: Gather the data
The datasets can be found here:
* https://www.kaggle.com/datasets/aemreusta/brfss-2020-survey-data
* https://www.kaggle.com/datasets/sakinak/behavioral-risk-factor-surveillance-survey-201619

In [ ]:
#@title Create Spark entry points
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [ ]:
sc = SparkContext(conf=SparkConf())
spark = SparkSession(sparkContext=sc)

In [ ]:
#@title Import Spark Mlib libraries
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import MultilayerPerceptronClassifier

# Step 3: Prepare data for consumption

In [ ]:
#@title Mount content to drive for kaggle data download
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download aemreusta/brfss-2020-survey-data

In [ ]:
! kaggle datasets download sakinak/behavioral-risk-factor-surveillance-survey-201619

In [ ]:
!ls

In [ ]:
!unzip brfss-2020-survey-data.zip

In [ ]:
!unzip behavioral-risk-factor-surveillance-survey-201619.zip

In [ ]:
from subprocess import check_output
print('-'*10, 'Files', '-'*10)
print(check_output(['ls', './']).decode('utf8'))

## About the BRFSS dataset and Prediction task

The Behavioral Risk Factor Surveillance System (BRFSS) is a collaborative project between all of the states in the United States and participating US territories and the Centers for Disease Control and Prevention (CDC).

BRFSS’s objective is to collect uniform state-specific data on health risk behaviors, chronic diseases and conditions, access to health care, and use of preventive health services related to the leading causes of death and disability in the United States. BRFSS conducts both landline and mobile phone-based surveys with individuals over the age of 18. General factors assessed by the BRFSS in 2020 included health status and healthy days, exercise, insufficient sleep, chronic health conditions, oral health, tobacco use, cancer screenings, and access to healthcare.

The aim of this project is to build a model with relatively high accuracy and AUC that could serve as an decision aid for those at high risk of developing lung cancer.

The data contains information about 401958 unique survey participant. As a result of my research to select the ones related to coronary artery disease among a total of 279 different features. Each example in the dataset contains the following demographic data for a set of individuals

### Categorical Features
*   `_AGE65YR`: The age of the individual in years two-level categories `18 <= AGE <= 64`: `1` and `65 <= AGE <= 99`:`2`
*   `SEXVAR`: Sex of Respondent `Male: 1` and `Female: 2`
*   `_BMI5CAT`:  Four-categories of Body Mass Index (BMI)`_BMI5 < 1850: Underweight` ; `1850 <= _BMI5 < 2500: Normal`;`2500 <= _BMI5 < 3000: Overweight`;`3000 <= _BMI5 < 9999: Obese`
*   `GENHLTH`: Health status: Would you say that in general your health is: `1: Excellent`; `2: Very good` ; `3: Good` ; `4: Fair` ; `5: Poor`
*   `SMOKE100`: Have you smoked at least 100 cigarettes in your entire life? [Note: 5 packs = 100 cigarettes] `1: Yes` ; `2: No`
*   `_SMOKER3`: Four-level smoker status: Everyday smoker: `1`, Someday smoker: `2`, Former smoker: `3`, Non-smoker: `4`

### Lung Cancer (Features) Screening Section
*   `LCSFIRST`: How old were you when you first started to smoke cigarettes regularly. `Value 1-100 in years`
*   `LCSLAST`: How old were you when you last smoked cigarettes regularly? `Value 1-100 in years`
*   `LCSNUMCG`: On average, when you smoke/smoked regularly, about how many cigarettes do/did you usually smoke each 
day? `Value 1-300 in number of cigarettes`
*   `LCSCTSCN`: In the last 12 months, did you have a CT or CAT scan? Example include: `Yes, to check for lung cancer`, `No (did not have a CT scan`, `Had a CT scan, but for other reason`.
*   `CNCRTYP1`:  What type of cancer was it? (If Response = 2 (Two) or 3 (Three or more), ask: “With your most recent 
diagnoses of cancer, what type of cancer was it?”). Examples include: `Lung cancer: 24`, `Others: 1-30`
*   `STOPSMK2`:  During the past 12 months, have you stopped smoking for one day or longer because you were trying to quit smoking? `Yes: 1` or `No: 2`.
*   `ECIGARET`: Have you ever used an e-cigarette or other electronic vaping product, even just one time, in your entire life? `Yes: 1` or `No: 2`.
*   `ECIGNOW`: Do you now use e-cigarettes or other electronic vaping products every day, some days, or not at all? `Every day: 1`; `Some days: 2` or `Not at all: 3`
* `ASTHMA3`: (Ever told) (you had) asthma? `Yes: 1` or `No: 2`.
### Prediction Task
The prediction task is to **early predict whether a person have the high risk of lung cancer.**

### Label
*   `CNCRTYP1`: What type of cancer (lung cancer = 24)





In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from matplotlib import rcParams
from sklearn.model_selection import train_test_split
import seaborn as sns

# The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

from google.colab import widgets
# For facets
from IPython.core.display import display, HTML
import base64
!pip install facets-overview==1.0.0
from facets_overview.feature_statistics_generator import FeatureStatisticsGenerator

In [ ]:
# load packages
import sys
print('Python version: {}'. format(sys.version))

import pandas as pd
print('Python version: {}'. format(pd.__version__))

import matplotlib
print('matplotlib version: {}'. format(matplotlib.__version__))

import numpy as np
print('numpy version: {}'. format(np.__version__))

import scipy as sp
print('scipy version: {}'. format(sp.__version__))

import IPython
from IPython import display # pretty printing of dataframe in Jupyter notebook
print('IPython version: {}'. format(IPython.__version__))

import pyspark
print('Apache Spark Pyspark version: {}'. format(pyspark.__version__)) # pyspark version

# misc libraries
import random
import time

# ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

In [ ]:
#@title Data Integration
from pyspark.sql.types import *

data_2020 = spark.read.csv('./brfss2020.csv', inferSchema=True, header=True)

# preview the data
# data type
print('-'*10, 'data types', '-'*10)
pd.DataFrame(data_2020.dtypes)

In [ ]:
data_2020F = data_2020.select('SEXVAR', '_AGE65YR', '_BMI5CAT', 'GENHLTH', 'CHECKUP1', #frequency of checkup
                              'EXERANY2', #Exercise in Past 30 Days
                              'SMOKE100', '_SMOKER3',
                              'STOPSMK2', #try to quit smoking
                              'LASTSMK2',#time since quitting
                              #'LCSFIRST', 'LCSLAST', 'LCSNUMCG',
                              'ECIGARET', 'ECIGNOW',
                              'LCSCTSCN',
                              'CHCCOPD2',
                   'ASTHMA3','CVDSTRK3', #ever had a stroke
                              'CVDCRHD4', #Ever Diagnosed with Angina or Coronary Heart Disease
                              'CNCRTYP1')
data_2020F.show()

In [ ]:
from pyspark.sql.functions import col
data_2020F = data_2020F.withColumn("SEXVAR", col('SEXVAR').cast(IntegerType()))\
          .withColumn("_AGE65YR", col('_AGE65YR').cast(IntegerType()))\
          .withColumn("_BMI5CAT", col('_BMI5CAT').cast(IntegerType()))\
          .withColumn("GENHLTH", col('GENHLTH').cast(IntegerType()))\
          .withColumn("CHECKUP1", col('CHECKUP1').cast(IntegerType()))\
          .withColumn("EXERANY2", col('EXERANY2').cast(IntegerType()))\
          .withColumn("SMOKE100", col('SMOKE100').cast(IntegerType()))\
          .withColumn("_SMOKER3", col('_SMOKER3').cast(IntegerType()))\
          .withColumn("STOPSMK2", col('STOPSMK2').cast(IntegerType()))\
          .withColumn("LASTSMK2", col('LASTSMK2').cast(IntegerType()))\
          .withColumn("ECIGARET", col('ECIGARET').cast(IntegerType()))\
          .withColumn("ECIGNOW", col('ECIGNOW').cast(IntegerType()))\
          .withColumn("LCSCTSCN", col('LCSCTSCN').cast(IntegerType()))\
          .withColumn("CHCCOPD2", col('CHCCOPD2').cast(IntegerType()))\
          .withColumn("ASTHMA3", col('ASTHMA3').cast(IntegerType()))\
          .withColumn("CVDSTRK3", col('CVDSTRK3').cast(IntegerType()))\
          .withColumn("CVDCRHD4", col('CVDCRHD4').cast(IntegerType()))\
          .withColumn("CNCRTYP1", col('CNCRTYP1').cast(IntegerType()))

In [ ]:
#401958
data_2020F = data_2020F.na.drop(how='any', thresh=None, subset=("GENHLTH", "CHECKUP1", "EXERANY2", "SMOKE100", "CVDSTRK3", "CVDCRHD4", "CHCCOPD2"))#"_BMI5CAT", , "ASTHMA3", "CHCCOPD2", "LCSFIRST","LCSLAST","LCSNUMCG","LCSCTSCN", "LASTSMK2", "STOPSMK2", "ECIGNOW"))
data_2020F.count()

In [ ]:
print('Columns with null values:')
print('-'*25)
data_2020F.select([eval('data_2020F.' + x + '.isNull().cast("int").alias("' + x + '")') for x in data_2020F.columns]).\
    groupBy().sum().toPandas()

In [ ]:
data_2020F.filter(data_2020F.CNCRTYP1 ==24.0).count()

In [ ]:
data_2020F.count()

In [ ]:
data_2020F.printSchema

In [ ]:
#@title User Defined Functions (UDF) for prediction label

from pyspark.sql.functions import udf
from pyspark.sql.types import *
y_udf = udf(lambda y: 1 if y == 24 else 0, StringType())


x_age = udf(lambda x: 0 if (x==1 or x==3) else 1, StringType())
x_bmi = udf(lambda x: x if (x==1 or x==2 or x==3 or x==4) else 2, StringType())
x_health = udf(lambda x: x if (x==1 or x==2 or x==3 or x==4 or x==5) else 0, StringType())
x_checkup = udf(lambda x: x if (x==1 or x==2 or x==3 or x==4 or x==8) else 0, StringType())
x_smoke3 = udf(lambda x: x if (x==1 or x==2 or x==3 or x==4) else 4, StringType())
x_lastsmk = udf(lambda x: x if (x==1 or x==2 or x==3 or x==4 or x==5 or x==6 or x==7 or x==8) else 0, StringType())
x_ecig = udf(lambda x: x if (x==1 or x==2 or x==3) else 0, StringType())
x_CT = udf(lambda x: x if (x==1 or x==3) else 0, StringType())
x_bool = udf(lambda x: 1 if (x==1) else 0, StringType())

processed_2020 = data_2020F.withColumn("Gender", x_bool('SEXVAR')).drop("SEXVAR")\
                    .withColumn("Age65", x_age('_AGE65YR')).drop("_AGE65YR")\
                    .withColumn("BMI", x_bmi('_BMI5CAT')).drop("_BMI5CAT")\
                    .withColumn("GeneralHealth", x_health('GENHLTH')).drop("GENHLTH")\
                    .withColumn("CheckupFreq", x_checkup('CHECKUP1')).drop("CHECKUP1")\
                    .withColumn("Exercise", x_bool('EXERANY2')).drop("EXERANY2")\
                    .withColumn("Smoked100", x_bool('SMOKE100')).drop("SMOKE100")\
                    .withColumn("SmokerStatus", x_smoke3('_SMOKER3')).drop("_SMOKER3")\
                    .withColumn("StopSmoking", x_bool('STOPSMK2')).drop("STOPSMK2")\
                    .withColumn("TimeSinceQuitSmk", x_smoke3('LASTSMK2')).drop("LASTSMK2")\
                    .withColumn("EverUsedEcig", x_bool('ECIGARET')).drop("ECIGARET")\
                    .withColumn("EcigLevel", x_ecig('ECIGNOW')).drop("ECIGNOW")\
                    .withColumn("HasCTScan", x_CT('LCSCTSCN')).drop("LCSCTSCN")\
                    .withColumn("HasChronicDisease", x_bool('CHCCOPD2')).drop("CHCCOPD2")\
                    .withColumn("HasAsthma", x_bool('ASTHMA3')).drop("ASTHMA3")\
                    .withColumn("HadStroke", x_bool('CVDSTRK3')).drop("CVDSTRK3")\
                    .withColumn("HadHeartDisease", x_bool('CVDCRHD4')).drop("CVDCRHD4")\
                    .withColumn("HasLungCancer", y_udf('CNCRTYP1')).drop("CNCRTYP1")

In [ ]:
processed_2020.groupBy(processed_2020.HasLungCancer).count().show()
# Data is unbalanced :)

In [ ]:
processed_2020.show()

In [ ]:
processed_2020.count()

In [ ]:
#Imputation
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols = ['FirstSmokedAge', 'LastSmokedAge', 'AvgNumCigADay'],
    outputCols = ['FirstSmokedAge', 'LastSmokedAge', 'AvgNumCigADay']
).setStrategy("mean")

In [ ]:
processed_2020 = imputer.fit(processed_2020).transform(processed_2020)

In [ ]:
processed_2020.show(5)

In [ ]:
column_subset = [col_ for col_ in processed_2020.columns]
for col_ in column_subset:
    temp_col = processed_2020.groupBy(col_).count()
    temp_col = temp_col.dropna(subset=col_)
    frequent_category = temp_col.orderBy(
                        temp_col['count'].desc()).show()

column_subset

In [ ]:
processed_2020.write.option("header",True).csv("data2020")

In [ ]:
#@title Data Integration 2
data_2017 = spark.read.csv('./2017.csv', inferSchema=True, header=True)

# preview the data
# data type
print('-'*10, 'data types', '-'*10)
pd.DataFrame(data_2017.dtypes)

In [ ]:
data_2017F = data_2017.select('SEX', '_AGE65YR', '_BMI5CAT', 'GENHLTH', 'CHECKUP1', #frequency of checkup
                              'EXERANY2', #Exercise in Past 30 Days
                              'SMOKE100', '_SMOKER3',
                              'STOPSMK2', #try to quit smoking
                              'LASTSMK2',#time since quitting
                              #'LCSFIRST', 'LCSLAST', 'LCSNUMCG',
                              'ECIGARET', 'ECIGNOW',
                              'LCSCTSCN',
                              'CHCCOPD1',
                   'ASTHMA3','CVDSTRK3', #ever had a stroke
                              'CVDCRHD4', #Ever Diagnosed with Angina or Coronary Heart Disease
                              'CNCRTYP1')
data_2017F.show(6)

In [ ]:
#401958
#data_2017F = data_2017F.na.drop(how="any")
data_2017F.count()

In [ ]:
from pyspark.sql.functions import col
data_2017F = data_2017F.withColumn("SEX", col('SEX').cast(IntegerType()))\
          .withColumn("_AGE65YR", col('_AGE65YR').cast(IntegerType()))\
          .withColumn("_BMI5CAT", col('_BMI5CAT').cast(IntegerType()))\
          .withColumn("GENHLTH", col('GENHLTH').cast(IntegerType()))\
          .withColumn("CHECKUP1", col('CHECKUP1').cast(IntegerType()))\
          .withColumn("EXERANY2", col('EXERANY2').cast(IntegerType()))\
          .withColumn("SMOKE100", col('SMOKE100').cast(IntegerType()))\
          .withColumn("_SMOKER3", col('_SMOKER3').cast(IntegerType()))\
          .withColumn("STOPSMK2", col('STOPSMK2').cast(IntegerType()))\
          .withColumn("LASTSMK2", col('LASTSMK2').cast(IntegerType()))\
          .withColumn("ECIGARET", col('ECIGARET').cast(IntegerType()))\
          .withColumn("ECIGNOW", col('ECIGNOW').cast(IntegerType()))\
          .withColumn("LCSCTSCN", col('LCSCTSCN').cast(IntegerType()))\
          .withColumn("CHCCOPD1", col('CHCCOPD1').cast(IntegerType()))\
          .withColumn("ASTHMA3", col('ASTHMA3').cast(IntegerType()))\
          .withColumn("CVDSTRK3", col('CVDSTRK3').cast(IntegerType()))\
          .withColumn("CVDCRHD4", col('CVDCRHD4').cast(IntegerType()))\
          .withColumn("CNCRTYP1", col('CNCRTYP1').cast(IntegerType()))

In [ ]:
#450016
data_2017F = data_2017F.na.drop(how='any', thresh=None, subset=("_BMI5CAT","_AGE65YR", "GENHLTH", "CHECKUP1", "EXERANY2", "SMOKE100", "_SMOKER3", "ECIGARET", "CHCCOPD1", "ASTHMA3", "CVDSTRK3", "CVDCRHD4"))#"_BMI5CAT",, "ASTHMA3", "CHCCOPD2", "LCSFIRST","LCSLAST","LCSNUMCG","LCSCTSCN", "LASTSMK2", "STOPSMK2", "ECIGNOW"))
data_2017F.count()

In [ ]:
print('Columns with null values:')
print('-'*25)
data_2017F.select([eval('data_2017F.' + x + '.isNull().cast("int").alias("' + x + '")') for x in data_2017F.columns]).\
    groupBy().sum().toPandas()

In [ ]:
data_2017F.count()

In [ ]:
#@title User Defined Functions (UDF) for prediction label

from pyspark.sql.functions import udf
from pyspark.sql.types import *

processed_2017 = data_2017F.withColumn("Gender", x_bool('SEX')).drop("SEX")\
                    .withColumn("Age65", x_age('_AGE65YR')).drop("_AGE65YR")\
                    .withColumn("BMI", x_bmi('_BMI5CAT')).drop("_BMI5CAT")\
                    .withColumn("GeneralHealth", x_health('GENHLTH')).drop("GENHLTH")\
                    .withColumn("CheckupFreq", x_checkup('CHECKUP1')).drop("CHECKUP1")\
                    .withColumn("Exercise", x_bool('EXERANY2')).drop("EXERANY2")\
                    .withColumn("Smoked100", x_bool('SMOKE100')).drop("SMOKE100")\
                    .withColumn("SmokerStatus", x_smoke3('_SMOKER3')).drop("_SMOKER3")\
                    .withColumn("StopSmoking", x_bool('STOPSMK2')).drop("STOPSMK2")\
                    .withColumn("TimeSinceQuitSmk", x_smoke3('LASTSMK2')).drop("LASTSMK2")\
                    .withColumn("EverUsedEcig", x_bool('ECIGARET')).drop("ECIGARET")\
                    .withColumn("EcigLevel", x_ecig('ECIGNOW')).drop("ECIGNOW")\
                    .withColumn("HasCTScan", x_CT('LCSCTSCN')).drop("LCSCTSCN")\
                    .withColumn("HasChronicDisease", x_bool('CHCCOPD1')).drop("CHCCOPD1")\
                    .withColumn("HasAsthma", x_bool('ASTHMA3')).drop("ASTHMA3")\
                    .withColumn("HadStroke", x_bool('CVDSTRK3')).drop("CVDSTRK3")\
                    .withColumn("HadHeartDisease", x_bool('CVDCRHD4')).drop("CVDCRHD4")\
                    .withColumn("HasLungCancer", y_udf('CNCRTYP1')).drop("CNCRTYP1")

In [ ]:
processed_2017.show()

In [ ]:
processed_2017.groupBy(processed_2017.HasLungCancer).count().show()
# Data is unbalanced :)

In [ ]:
#Imputation
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols = ['FirstSmokedAge', 'LastSmokedAge', 'AvgNumCigADay'],
    outputCols = ['FirstSmokedAge', 'LastSmokedAge', 'AvgNumCigADay']
).setStrategy("mean")

In [ ]:
processed_2017 = imputer.fit(processed_2017).transform(processed_2017)

In [ ]:
processed_2017.show(5)

In [ ]:
column_subset = [col_ for col_ in processed_2020.columns]
for col_ in column_subset:
    temp_col = processed_2020.groupBy(col_).count()
    temp_col = temp_col.dropna(subset=col_)
    frequent_category = temp_col.orderBy(
                        temp_col['count'].desc()).show()

column_subset

In [ ]:
#@title Data Integration 3
data_2018 = spark.read.csv('./2018.csv', inferSchema=True, header=True)

# preview the data
# data type
print('-'*10, 'data types', '-'*10)
pd.DataFrame(data_2018.dtypes)

In [ ]:
data_2018F = data_2018.select('SEX1', '_AGE65YR', '_BMI5CAT', 'GENHLTH', 'CHECKUP1', #frequency of checkup
                              'EXERANY2', #Exercise in Past 30 Days
                              'SMOKE100', '_SMOKER3',
                              'STOPSMK2', #try to quit smoking
                              'LASTSMK2',#time since quitting
                              #'LCSFIRST', 'LCSLAST', 'LCSNUMCG',
                              'ECIGARET', 'ECIGNOW',
                              'LCSCTSCN',
                              'CHCCOPD1',
                   'ASTHMA3','CVDSTRK3', #ever had a stroke
                              'CVDCRHD4', #Ever Diagnosed with Angina or Coronary Heart Disease
                              'CNCRTYP1')
data_2018F.show(6)

In [ ]:
from pyspark.sql.functions import col
data_2018F = data_2018F.withColumn("SEX1", col('SEX1').cast(IntegerType()))\
          .withColumn("_AGE65YR", col('_AGE65YR').cast(IntegerType()))\
          .withColumn("_BMI5CAT", col('_BMI5CAT').cast(IntegerType()))\
          .withColumn("GENHLTH", col('GENHLTH').cast(IntegerType()))\
          .withColumn("CHECKUP1", col('CHECKUP1').cast(IntegerType()))\
          .withColumn("EXERANY2", col('EXERANY2').cast(IntegerType()))\
          .withColumn("SMOKE100", col('SMOKE100').cast(IntegerType()))\
          .withColumn("_SMOKER3", col('_SMOKER3').cast(IntegerType()))\
          .withColumn("STOPSMK2", col('STOPSMK2').cast(IntegerType()))\
          .withColumn("LASTSMK2", col('LASTSMK2').cast(IntegerType()))\
          .withColumn("ECIGARET", col('ECIGARET').cast(IntegerType()))\
          .withColumn("ECIGNOW", col('ECIGNOW').cast(IntegerType()))\
          .withColumn("LCSCTSCN", col('LCSCTSCN').cast(IntegerType()))\
          .withColumn("CHCCOPD1", col('CHCCOPD1').cast(IntegerType()))\
          .withColumn("ASTHMA3", col('ASTHMA3').cast(IntegerType()))\
          .withColumn("CVDSTRK3", col('CVDSTRK3').cast(IntegerType()))\
          .withColumn("CVDCRHD4", col('CVDCRHD4').cast(IntegerType()))\
          .withColumn("CNCRTYP1", col('CNCRTYP1').cast(IntegerType()))

In [ ]:
#437436
data_2018F = data_2018F.na.drop(how='any', thresh=None, subset=("_BMI5CAT","_AGE65YR", "GENHLTH", "CHECKUP1", "EXERANY2", "SMOKE100", "_SMOKER3", "CHCCOPD1", "ASTHMA3", "CVDSTRK3", "CVDCRHD4"))#"_BMI5CAT",, "ASTHMA3", "CHCCOPD2", "LCSFIRST","LCSLAST","LCSNUMCG","LCSCTSCN", "LASTSMK2", "STOPSMK2", "ECIGNOW"))
data_2018F.count()

In [ ]:
print('Columns with null values:')
print('-'*25)
data_2018F.select([eval('data_2018F.' + x + '.isNull().cast("int").alias("' + x + '")') for x in data_2018F.columns]).\
    groupBy().sum().toPandas()

In [ ]:
data_2018F.count()

In [ ]:
data_2018F.printSchema()

In [ ]:
#@title User Defined Functions (UDF) for prediction label

from pyspark.sql.functions import udf
from pyspark.sql.types import *
y_udf = udf(lambda y: 1 if y == 24 else 0, StringType())

processed_2018 = data_2018F.withColumn("Gender", x_bool('SEX1')).drop("SEX1")\
                    .withColumn("Age65", x_age('_AGE65YR')).drop("_AGE65YR")\
                    .withColumn("BMI", x_bmi('_BMI5CAT')).drop("_BMI5CAT")\
                    .withColumn("GeneralHealth", x_health('GENHLTH')).drop("GENHLTH")\
                    .withColumn("CheckupFreq", x_checkup('CHECKUP1')).drop("CHECKUP1")\
                    .withColumn("Exercise", x_bool('EXERANY2')).drop("EXERANY2")\
                    .withColumn("Smoked100", x_bool('SMOKE100')).drop("SMOKE100")\
                    .withColumn("SmokerStatus", x_smoke3('_SMOKER3')).drop("_SMOKER3")\
                    .withColumn("StopSmoking", x_bool('STOPSMK2')).drop("STOPSMK2")\
                    .withColumn("TimeSinceQuitSmk", x_smoke3('LASTSMK2')).drop("LASTSMK2")\
                    .withColumn("EverUsedEcig", x_bool('ECIGARET')).drop("ECIGARET")\
                    .withColumn("EcigLevel", x_ecig('ECIGNOW')).drop("ECIGNOW")\
                    .withColumn("HasCTScan", x_CT('LCSCTSCN')).drop("LCSCTSCN")\
                    .withColumn("HasChronicDisease", x_bool('CHCCOPD1')).drop("CHCCOPD1")\
                    .withColumn("HasAsthma", x_bool('ASTHMA3')).drop("ASTHMA3")\
                    .withColumn("HadStroke", x_bool('CVDSTRK3')).drop("CVDSTRK3")\
                    .withColumn("HadHeartDisease", x_bool('CVDCRHD4')).drop("CVDCRHD4")\
                    .withColumn("HasLungCancer", y_udf('CNCRTYP1')).drop("CNCRTYP1")
                    

In [ ]:
processed_2018.show()

In [ ]:
processed_2018.groupBy(processed_2018.HasLungCancer).count().show()
# Data is unbalanced :)

In [ ]:
processed_2018.write.option("header",True).csv("df_test")

In [ ]:
processed_2018.show()

In [ ]:
#Imputation
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols = ['FirstSmokedAge', 'LastSmokedAge', 'AvgNumCigADay'],
    outputCols = ['FirstSmokedAge', 'LastSmokedAge', 'AvgNumCigADay']
).setStrategy("mean")

In [ ]:
processed_2018 = imputer.fit(processed_2018).transform(processed_2018)

In [ ]:
processed_2018.show(5)

In [ ]:
column_subset = [col_ for col_ in processed_2020.columns]
for col_ in column_subset:
    temp_col = processed_2020.groupBy(col_).count()
    temp_col = temp_col.dropna(subset=col_)
    frequent_category = temp_col.orderBy(
                        temp_col['count'].desc()).show()

column_subset

In [ ]:
#437436
data_2018F = data_2018F.na.drop(how="any")
data_2018F.count()

In [ ]:
data_2018F.show(6)

In [ ]:
data_2018F.groupBy(data_2018F.CNCRTYP1==24).count().show()
# Data is unbalanced :)

In [ ]:
test_data = data_2018F.filter(data_2018F.CNCRTYP1 == 24)
test_data.show()

In [ ]:
#@title Data Preprocessing
df_train= processed_2020.unionByName(processed_2017)

# preview the data
# data type
print('-'*10, 'data types', '-'*10)
pd.DataFrame(data.dtypes)

In [ ]:
df_train.groupBy(df_train.HasLungCancer).count().show()


In [ ]:
df_train.write.option("header",True).csv("df_train")

In [ ]:
#@title Data Integration 4
data_2019 = spark.read.csv('./2019.csv', inferSchema=True, header=True)

# preview the data
# data type
print('-'*10, 'data types', '-'*10)
pd.DataFrame(data_2019.dtypes)

In [ ]:
data_2019F = data_2019.select('SEXVAR', '_AGE65YR', '_BMI5CAT', 'GENHLTH', 'SMOKE100', '_SMOKER3',
                  'LCSFIRST', 'LCSLAST', 'LCSNUMCG', 'LCSCTSCN', 'CNCRTYP1',
                  'STOPSMK2', 'ASTHMA3', 'CHCCOPD2') #'ECIGARET',  'ECIGNOW'
data_2019F.show(6)

In [ ]:
print('Columns with null values:')
print('-'*25)
data_2019F.select([eval('data_2019F.' + x + '.isNull().cast("int").alias("' + x + '")') for x in data_2019F.columns]).\
    groupBy().sum().toPandas()

In [ ]:
#data_2019F = data_2019F.na.drop(how="any")
data_2019F.count()

In [ ]:
data_2019F.show(6)

In [ ]:
data_2019F.filter(data_2019F.CNCRTYP1 != 'NA').count()

In [ ]:
data_2019F.printSchema()

In [ ]:
#@title User Defined Functions (UDF) for prediction label

from pyspark.sql.functions import udf
from pyspark.sql.types import *
y_udf = udf(lambda y: '1' if y == '24' else '0', StringType())
x_udf = udf(lambda x: '0' if (x=='NA' or x=='7' or x=='77' or x=='777' or x=='9' or x=='99' or x=='888' or x=='999') else x, StringType())

processed_2019 = data_2019F.withColumn("Gender", x_udf('SEXVAR')).drop("SEXVAR")\
                    .withColumn("Age65", x_udf('_AGE65YR')).drop("_AGE65YR")\
                    .withColumn("GeneralHealth", x_udf('GENHLTH')).drop("GENHLTH")\
                    .withColumn("Smoked100", x_udf('SMOKE100')).drop("SMOKE100")\
                    .withColumn("SmokerStatus", x_udf('_SMOKER3')).drop("_SMOKER3")\
                    .withColumn("FirstSmokedAge", x_udf('LCSFIRST')).drop("LCSFIRST")\
                    .withColumn("LastSmokedAge", x_udf('LCSLAST')).drop("LCSLAST")\
                    .withColumn("AvgNumCigADay", x_udf('LCSNUMCG')).drop("LCSNUMCG")\
                    .withColumn("HasCTScan", x_udf('LCSCTSCN')).drop("LCSCTSCN")\
                    .withColumn("StopSmoking", x_udf('STOPSMK2')).drop("STOPSMK2")\
                    .withColumn("HasAsthma", x_udf('ASTHMA3')).drop("ASTHMA3")\
                    .withColumn("HasChronicDisease", x_udf('CHCCOPD2')).drop("CHCCOPD2")\
                    .withColumn("HasLungCancer", y_udf('CNCRTYP1')).drop("CNCRTYP1")
                    

In [ ]:
processed_2019.printSchema()

In [ ]:
processed_2019.groupBy(processed_2019.HasLungCancer).count().show()
# Data is unbalanced :)

In [ ]:
#@title Data Preprocessing
data = processed_2020.unionByName(processed_2017)

# preview the data
# data type
print('-'*10, 'data types', '-'*10)
pd.DataFrame(data.dtypes)

In [ ]:
data_raw = data.unionByName(processed_2018)

# preview the data
# data type
print('-'*10, 'data types', '-'*10)
pd.DataFrame(data_raw.dtypes)

In [ ]:
# Define schema explitcitly
from pyspark.sql.types import *
data_raw.columns

In [ ]:
data_raw.groupBy(data_raw.HasLungCancer).count().show()
# Data is unbalanced :)

In [ ]:
# data summary
print('-'*10, 'data summary', '-'*10)
data_raw.describe().toPandas()

In [ ]:
data_raw_copy = data_raw.select('*')

In [ ]:
data_raw_copy = data_raw_copy.where(data_raw.Age65 != '0')
data_raw_copy = data_raw_copy.where(data_raw.Gender != '0')

In [ ]:
from pyspark.sql.functions import *


                    #.withColumn("LastSmokedAge", regexp_replace('LastSmokedAge', '0', '30'))\
                    #.withColumn("AvgNumCigADay", regexp_replace('Smoked100', '0', '2'))\
data_raw_copy = data_raw_copy.withColumn("BMI", regexp_replace('BMI', '0', '2'))\
                    .withColumn("GeneralHealth", regexp_replace('GeneralHealth', '0', '2'))\
                    .withColumn("Smoked100", regexp_replace('Smoked100', '0', '2'))\
                    .withColumn("SmokerStatus", regexp_replace('SmokerStatus', '0', '4'))\
                    .withColumn("FirstSmokedAge", regexp_replace('FirstSmokedAge', '0', '18'))\
                    .withColumn("HasCTScan", regexp_replace('HasCTScan', '0', '2'))\
                    .withColumn("StopSmoking", regexp_replace('StopSmoking', '0', '2'))\
                    .withColumn("HasAsthma", regexp_replace('HasAsthma', '0', '2'))\
                    .withColumn("HasChronicDisease", regexp_replace('HasChronicDisease', '0', '2'))

In [ ]:
data_raw_copy = data_raw_copy.filter(data_raw_copy.GeneralHealth !=0)

In [ ]:
column_subset = [col_ for col_ in data_raw_copy.columns if data_raw_copy.select(col_).dtypes!="string"]
for col_ in column_subset:
    temp_col = data_raw_copy.groupBy(col_).count()
    temp_col = temp_col.dropna(subset=col_)
    frequent_category = temp_col.orderBy(
                        temp_col['count'].desc()).show()

column_subset

In [ ]:
column_subset = [col_ for col_ in data_raw.columns if data_raw.select(col_).dtypes!="string"]
for col_ in column_subset:
    temp_col = data_raw.groupBy(col_).count()
    temp_col = temp_col.dropna(subset=col_)
    frequent_category = temp_col.orderBy(
                        temp_col['count'].desc()).show()

column_subset

In [ ]:
column_subset = [col_ for col_ in data_raw.columns if data_raw.select(col_).dtypes!="string"]
for col_ in column_subset:
    temp_col = data_raw.groupBy(col_).count()
    temp_col = temp_col.dropna(subset=col_)
    frequent_category=temp_col.orderBy(
                     temp_col['count'].desc()).collect()[0][0]
    data_raw = data_raw.replace(frequent_category, subset=col_)
data_raw.show()

In [ ]:
data_raw.write.option("header",True).csv("final_data")

# Step 4: Decision Tree Classification with PySpark

In [ ]:
#@title Process categorical columns
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, Imputer, BucketedRandomProjectionLSH,VectorSlicer
from pyspark.sql.window import Window
from pyspark.ml.linalg import Vectors,VectorUDT
from pyspark.sql.functions import array, create_map, struct
from pyspark.ml import Pipeline

# categorical columns
categorical_columns = data_raw.columns[0:12]

In [ ]:
#@title Build StringIndexer stages
stringindexer_stages = [StringIndexer(inputCol=c, outputCol='strindexed_' + c) for c in categorical_columns]
# encode label column and add it to stringindexer_stages
stringindexer_stages += [StringIndexer(inputCol='HasLungCancer', outputCol='label')]

In [ ]:
#@title Build OneHotEncoder stages
onehotencoder_stages = [OneHotEncoder(inputCol='strindexed_' + c, outputCol='onehot_' + c) for c in categorical_columns]

In [ ]:
#@title Build VectorAssembler stage
feature_columns = ['onehot_' + c for c in categorical_columns]
vectorassembler_stage = VectorAssembler(inputCols=feature_columns, outputCol='features') 

In [ ]:
#@title Build Pipeline model
# all stages
all_stages = stringindexer_stages + onehotencoder_stages + [vectorassembler_stage]
pipeline = Pipeline(stages=all_stages)

In [ ]:
#@title Fit pipeline model
pipeline_model = pipeline.fit(data_raw)

In [ ]:
#@title Transform data
final_columns = feature_columns + ['features', 'label']
df_raw = pipeline_model.transform(data_raw).\
            select(final_columns)
            
df_raw.show(5)

In [ ]:
#@title Split data into traning and test sets
training, test = df_raw.randomSplit([0.8, 0.2], seed=1234)

In [ ]:
##@title Data Imputing
#    inputCols = ['Age of Employee', 'Experience (in years)', 'Salary (per month - $)'],
#    outputCols = ["{}_imputed".format(a) for a in ['Age of Employee', 'Experience (in years)', 'Salary (per month - $)']]
#).setStrategy("mean")

In [ ]:
training.printSchema() #'onehot_SEXVAR', 'onehot__AGE65YR', 'onehot__BMI5CAT', 'onehot_GENHLTH',
                       #     'onehot_SMOKE100', 'onehot__SMOKER3', 'onehot_LCSFIRST', 'onehot_LCSLAST',
                       #     'onehot_LCSNUMCG', 'onehot_LCSCTSCN', 'onehot_STOPSMK2', 'onehot_ASTHMA3', 

In [ ]:
#@title Data balancing using SMOTE
#K-nearest neighbor algorithm to simulate the minority sample
from imblearn.over_sampling import SMOTE

features = training.select(['features']).toPandas()

labels = training.select('label').toPandas()

In [ ]:
sm = SMOTE(sampling_strategy = 'not majority', k_neighbors = 50, random_state = 42)

features, labels = sm.fit_resample(features, labels)

In [ ]:
features['label'] = labels.values
features = spark.createDataFrame(features)

In [ ]:
#@title Build cross validation 
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol='features', labelCol='label')

In [ ]:
#@title Parameter grid
from pyspark.ml.tuning import ParamGridBuilder
param_grid = ParamGridBuilder().\
    addGrid(dt.maxDepth, [2,3,4,5]).\
    build()

In [ ]:
#@title Evaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", metricName="areaUnderROC")

In [ ]:
#@title Cross-validation model
from pyspark.ml.tuning import CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=4)

In [ ]:
#@title Fit cross validation model
cv_model = cv.fit(df_raw)

In [ ]:
show_columns = ['features', 'label', 'prediction', 'rawPrediction', 'probability']

In [ ]:
#@title Prediction on training data
pred_training_cv = cv_model.transform(training)
pred_training_cv.select(show_columns).show(5, truncate=False)

In [ ]:
#@title Prediction on test data
pred_test_cv = cv_model.transform(test)
pred_test_cv.select(show_columns).show(5, truncate=False)

In [ ]:
#@title Confusion matrix
label_and_pred = cv_model.transform(df_raw).select('label', 'prediction')
label_and_pred.rdd.zipWithIndex().countByKey()

In [ ]:
print('The best MaxDepth is:', cv_model.bestModel._java_obj.getMaxDepth())